In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

### Simple least square regression

In [2]:
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3


W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b


sess = tf.Session()
sess.run(tf.initialize_all_variables())

loss = tf.reduce_mean(tf.square(y_data - y))
optimizer = tf.train.GradientDescentOptimizer(0.09)
train = optimizer.minimize(loss)

for i in range(1000):
    sess.run(train)

W.eval(sess), b.eval(sess)

### SVM example

In [565]:
x_data = ((np.random.rand(100) - 0.5)*6.56).astype(np.float32)
y_data = (2*(x_data * 0.1459 + 0.04567 > 0) - 1.).astype(np.float32)


In [581]:
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
yp = W*x_data + b
loss =   tf.reduce_mean(tf.maximum(0., (1 - tf.mul(y_data, yp) ))) + tf.reduce_sum(tf.square(W))

In [593]:
loss =   tf.reduce_mean(tf.maximum(0., (1 - tf.mul(y_data, yp) ))) 

In [582]:
optimizer = tf.train.GradientDescentOptimizer(0.09)
train = optimizer.minimize(loss)
sess = tf.Session()

In [585]:
sess.run(tf.initialize_all_variables())

In [599]:
train = optimizer.minimize(loss)
for i in range(10000):
    sess.run(train)

#### SVM on mnist

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [606]:
type(mnist[1])

tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [4]:
sess = tf.InteractiveSession()

In [6]:
tr = mnist[0]

In [611]:
tr.images

(55000, 784)

In [7]:
x_data = tr.images.astype(np.float32)
y_data = (tr.labels[:,0] * 2 - 1).astype(np.float32)

In [628]:
x_data.shape

(55000, 784)

In [633]:
tf.reduce_sum(W*x_data, 1).get_shape()

TensorShape([Dimension(55000)])

In [8]:
W = tf.Variable(tf.random_uniform([784], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
yp = tf.reduce_sum(W*x_data, 1)+b
loss =  tf.reduce_mean(tf.maximum(0., (1 - tf.mul(y_data, yp) ))) + 0.1*tf.reduce_sum(tf.square(W))

In [9]:
optimizer = tf.train.GradientDescentOptimizer(0.09)
train = optimizer.minimiz(e(loss)


In [11]:
sess.run(tf.initialize_all_variables())

In [12]:
for i in range(100):
    sess.run(train)

In [17]:
ts = mnist[2]
x_ts = ts.images.astype(np.float32)
y_ts = (ts.labels[:,0] * 2 - 1).astype(np.float32)

In [19]:
yp_ts = tf.reduce_sum(W*x_ts,1) + b
sess.run(tf.reduce_mean(tf.to_float(yp_ts*y_ts > 0)))

0.98020011

In [20]:
sess.close()

#### Multi Class SVM

In [7]:
sess = tf.InteractiveSession()

In [22]:
W = W = tf.Variable(tf.random_uniform([784, 10], -1.0, 1.0))
b = tf.Variable(tf.zeros([10]))


x_tr = tr.images.astype(np.float32)
y_tr = (tr.labels*2 - 1).astype(np.float32)

yp = tf.reduce_sum(tf.matmul(x_tr,W),0)+b

loss = tf.reduce_mean(tf.maximum(0., (1 - y_tr * yp ))) + tf.reduce_sum(W)

In [43]:
train = optimizer.minimize(loss)

sess.run(tf.initialize_all_variables())

Training on the entire set will not work, I got an error stating that the graph size cannot be more than 2G. So time to work on batches of data. Picking 1024 as the batch size

In [52]:
batch_size = 1024
W = W = tf.Variable(tf.random_uniform([784, 10], -1.0, 1.0))
b = tf.Variable(tf.zeros([10]))
x_batch = tf.placeholder(tf.float32, [None, 784])
y_batch = tf.placeholder(tf.float32, [None, 10])
yp = tf.matmul(x_batch,W)+b


In [87]:
reg_constant = 1
loss = tf.reduce_mean(tf.maximum(0., (1 - y_batch * yp ))) + tf.reduce_sum(tf.square(W))*reg_constant

In [88]:
sess.run(tf.initialize_all_variables())

In [89]:
optimizer = tf.train.GradientDescentOptimizer(0.1)

In [90]:
train = optimizer.minimize(loss)

In [57]:
tr = mnist[0]
x_tr = tr.images.astype(np.float32)
y_tr = (tr.labels*2 - 1).astype(np.float32)


In [97]:
for i in range(15000): # Run 1000 iterations on the data
    rng = np.random.randint(0, x_tr.shape[0], batch_size)
    xbtch = x_tr[rng,:]
    ybtch = y_tr[rng,:]
    sess.run(train, feed_dict={x_batch:xbtch, y_batch:ybtch})

##### measure accuracy

In [60]:
ts = mnist[2]
xts = ts.images.astype(np.float32)
yts = (ts.labels*2 - 1).astype(np.float32)


In [69]:
correct_prediction = tf.equal(tf.argmax(yp,1), tf.argmax(y_batch,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [98]:
sess.run(accuracy, feed_dict={x_batch:xts, y_batch:yts})

0.37979993

In [58]:
yp.get_shape()

TensorShape([Dimension(None), Dimension(10)])

In [1]:
38950*64

2492800